In [2]:
import pandas as pd
import numpy as np
import glob

In [3]:
import folium
import time, re
from tqdm import tqdm_notebook as tqdmn

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

---

---  
# BREVE INFORMACIÓN IMPORTANTE

## Explicación de cuanto tiempo y como fue el proceso de scraping real
Esta parte del proceso ha tardado 4 días aproximadamente en realizarse (este intervalo de tiempo se estima sin tener en cuenta el tiempo que se perdía cuando ocurrían las interrupciones abruptas).
Factor importante para continuar es que los registros que se consultaron fueron los aproximadamente ochenta mil registros que proporciono Eixos al inicio de este análisis.
Teniendo en cuenta que serian ochenta mil consultas, que el proceso de scraping necesitaba mucho tiempo de procesamiento y también de que el ordenador donde trabajaba no lo quería dejar de utilizar para dedicarlo a esta fase, por eso se optó por utilizar otro ordenador. Un ordenador viejo que se tenía guardado. A este ordenador se le instaló Ubuntu 18 y la distribución de Anaconda. La idea fue dedicar directamente este ordenador a que estuviera consultando este dataset que facilito Eixos.  
Y así fue, para esto también se tuvo que instalar el ChromeDriver para Linux, la librería Selenium, Folium y definitivamente el navegador Google Chrome en su versión 81. En cuanto a esto, se debe mencionar que toda la configuración en Linux se realizó por línea de comandos. Pero sin contratiempos, se fueron desarrollando bien las consultas.  
Cuando se refiere a la configuración por terminal se toma en cuenta el cambiar el Google Chrome de versión 81 a versión 80, cambiar la versión de Selenium en Anaconda, asignar al path el ChromeDriver o ir probando instalar distintas versiones de ChromeDriver, entre otras configuraciones que se realizaron en este ordenador que se dedicó a esta tarea.  
Esto hasta que después de consultar 25 mil registros un error no dejo seguir haciendo consultas. Este suceso se siguió repitiendo hasta el final de las consultas por lo que muchas veces las consultas fueron paradas abruptamente y por esta razón hay 19 data sets donde el primer número en el nombre de cada fichero CSV indica hasta el registro al que se consultó y el segundo número indica cuantos registros se obtuvieron satisfactoriamente y cuantos se quedaron guardados en dicho fichero.  
El célebre error fue este:

***TimeoutException: Message: timeout: Timed out receiving message from renderer: 16,270(Session info: headless chrome=81.0.4044.122)***

Este problema recurrente se consultó en foros como StackOverflow y si que aparecía y se aplicaron las medidas que se sugerían, y si que aliviaban las interrupciones del código de consulta pero al final siempre volvía a ocurrir. Una hipótesis es que este problema se debía a que al haber realizado muchas consultas desde el ordenador dedicado a esta tarea, de alguna manera su IP fue baneada o puesta en una Black list. Esta hipótesis tomaba fuerza pues al principio las consultas y los registros encontrados satisfactoriamente fueron en gran manera exitosos, pero después se volvió lento y constantemente interrumpido. Puede ser que esto se haya debido a que al principio no se ha utilizado ningún tipo de VPN. Luego si que se hizo el cambio y configurando en la terminal el servicio HideMyVPN el proceso si que avanzo mejor.  
Finalmente este proceso tomo varios días, aproximadamente dos semanas, entre las interrupciones, la consecución del código definitivo que retornara los parámetros necesarios que se querían y la modificación constante debido a los distintos errores que aparecían en el proceso. Aunque se mencionó el mensaje de error anterior, también los hubo muchos más pero de estos no vale la pena discutir pues fueron resueltos con facilidad.

---

# CAPTURA DE DATOS DE GOOGLE MAPS USANDO HERRAMIENTA CHROMEDRIVER

Esta es una de las actividades esenciales para continuar con el proyecto. Se utilizará la herramienta Chrome Driver. Esta herramienta se utiliza normalmente para la automatización de despliegue de pruebas para sitios web y aplicaciones. En este caso en específico lo que se realizó fue automatizar una serie de pasos de un proceso que consistía en:
1. Utilizar el buscador de Google maps para consultar cierta cadena de caracteres.
2. Capturar ciertos parámetros del fichero HTML que se despliega al momento de encontrar el sitio indicado.
3. Guardar esta información en un fichero CSV.

Para el primer paso, utilizado el fichero de registros que tenia Eixos, se construye un parametro que se buscará en el navegador cumpliendo ciertos condiciones. A continuacion se muestra como se realizo este paso:

In [20]:
df = pd.read_csv('../df_sup.csv', usecols=['title','street','number']).head(30)

In [21]:
df.loc[df['title'] == 'Anton Solé', 'title'] = 'Taller Anton Solé'
df['title'] = df['title'].str.replace('F.Espunes','Talleres Espuña')
df.loc[df['title'] == 'Herpa', 'title'] = 'Erpa'
df.loc[df['title'] == 'Emauto S.C', 'number'] = '40'
df.loc[df['title'] == 'Emauto S.C', 'title'] = 'Em Auto S.C.'
df.loc[df['title'] == 'Findal', 'number'] = '128' 
df.loc[df['title'] == 'Findal', 'street'] = 'CL PUIG CERDÁ'
df.loc[df['title'] == 'Tallers Ulibarri', 'title'] = 'Talleres Chicote'

Primero hay que cambiar ciertos caractéres a algo que el buscador pueda manipular, por eso se cambian ciertas reducciones a su version normal.

In [22]:
df['street'] = df['street'].str.replace('CL','Carrer')
df['street'] = df['street'].str.replace('PZ','Plaça')
df['street'] = df['street'].str.replace('PS','Passeig')

df['serch_o'] = df['title']+' '+df['street']+', '+df['number']

Los datos que nos importaría pasarle al buscador serán la construcción de: 
* El nombre del establecimiento **+** La direccion del establecimiento (Calle, avenida,placa... ) **+** El número.  
De esta manera cuando se pase este dato uno a uno en la consulta en el buscador sera posible conseguir la información pues esto es mas parecido a como un ser humano realizaría una busqueda.

In [23]:
df.head()

,title,number,street,serch_o
0,Garces Taller,78,Carrer ROCAFORT,"Garces Taller Carrer ROCAFORT, 78"
1,Taller Anton Solé,10,Plaça NAVAS,"Taller Anton Solé Plaça NAVAS, 10"
2,Auto.Carburación,1,Carrer TEODOR BONAPLATA,"Auto.Carburación Carrer TEODOR BONAPLATA, 1"
3,"M.F, Flomart",30,Passeig MONTJUIC,"M.F, Flomart Passeig MONTJUIC, 30"
4,Talleres Espuña,68,Passeig MONTJUIC,"Talleres Espuña Passeig MONTJUIC, 68"


Cuando ya se tiene construido nuestro elemento, lo siguiente será ir a buscar dicho elemento que se pasa por la herramienta de automatización ChromeDrive.
Los parámetros que nos interesaran son:
* full_name: El nombre completo del establecimiento, COMO SE ENCUENTRA EN EL SITIO DE GOOGLE MAPS.
* rating: El rating de valoraciones (5-0).
* total_ratings: El número de personas que han realizado una reseña
* landmark_cat: La categoría del establecimiento, SEGÚN LO CLASIFICA GOOGLE MAPS.
* description: Descripción del establecimiento si la hay (normalmente no hay descripción).
* address: La dirección del establecimiento, en la forma calle, avenida, plaza, pasaje... + número + código postal + ciudad
* hours: Cuando está disponible, se captura el horario de atención y su nivel de concurrencia. (puede variar en horario diario(04 h-21 h,05 h-22 h,06 h-23 h)) y por días abiertos, como por ejemplo abierto todos los días, cerrando los sábados y domingos o solo los domingos.
* lat: SEGÚN GOOGLE latitud del establecimiento.
* long: SEGÚN GOOGLE longitud del establecimiento.

In [26]:
options = webdriver.ChromeOptions()

options.add_argument('headless')

browser = webdriver.Chrome(options=options)

landmarks = df['serch_o']# PS.full_name+' '+PS.address


# We want to remove any '/' character in the names and addresses in the landmarks list (because they'll break URLs) :
landmarks = [i.replace('/', ' ') for i in landmarks]

# These are the empty lists we will populate with the extracted data :
full_name = []
rating = []
total_ratings = []
landmark_cat = []
description = []
address = []
hours = []
lat = []
long = []
    
# Here's the big loop iterating over the landmarks list :
for landmark in tqdmn(landmarks, leave=False, desc='2. Extracting the data') :
    
    # URL making :
    url = 'https://www.google.com/maps/search/' + landmark
    browser.get(url)

    # Waiting for the name of the landmark to load and be visible. If it fails, skip to next one :
    try :
        WebDriverWait(browser,30).until(EC.visibility_of_element_located((By.CLASS_NAME, "section-hero-header-title-title")))
    except (NoSuchElementException, TimeoutException) as e :
        continue
        
    # Extracting the data and putting it into the empty lists we defined earlier :
    try:
        full_name.append(browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]').text)
    except NoSuchElementException :
        full_name.append(np.nan)

    try:
        rating.append(browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[2]/div/div[1]/span[1]/span/span').text)
    except NoSuchElementException :
        rating.append(np.nan)

    try:
        total_ratings.append(browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[2]/div/div[1]/span[2]/span/span[1]/span[2]/span[1]/button').text)
    except NoSuchElementException:
        total_ratings.append(np.nan)

    try:
        landmark_cat.append(browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]/span[1]/button').text)
    except NoSuchElementException:
        landmark_cat.append(np.nan)
    
    try:
        description.append(browser.find_element_by_css_selector('div[class=section-editorial-quote]').text)
    except NoSuchElementException:
        description.append(np.nan
                          )
    try:
        address.append(browser.find_element_by_css_selector('div[data-tooltip="Copiar la dirección"]').text)
    except NoSuchElementException :
        address.append(np.nan)

    # Here we capture the popular hours for all 7 days starting with Sunday :
    try:
        hours.append([i.get_attribute('aria-label') for i in browser.find_elements_by_xpath("//*[contains(@aria-label, 'hora:')]")])
    except NoSuchElementException:
        hours.append(np.nan)
        
    try:
        coordinates = browser.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content')
        coordinates = coordinates.split('?center=')[1].split('&zoom=')[0].split('%2C')
        lat.append(coordinates[0])
        long.append(coordinates[1])
    except NoSuchElementException:
        lat.append(np.nan)
        long.append(np.nan)

# Closing the Chrome window
browser.close()

Hay muchas maneras de buscar los datos que interesan usando esta herramienta, se debe usar el fichero HTML para ir buscando contenedores u objetos. Es importante tener un conocimiento mínimo de HTML para realizar esta etapa. Sobre todo es para poder encontrar de manera específica los parámetros que nos interesan.  
Además es de reconocer la potencia de esta herramienta pues se pueden definir diversos pasos para que automáticamente dadas algunas condiciones pueda capturar más información navegando o escogiendo entre una lista de elementos al momento de realizar una búsqueda.  
Una perspectiva interesante para este proyecto seria guardar las reseñas que los usuarios ha dejado en cada comercio y realizar un análisis de NLP, solo por mencionar.

In [27]:
HL = pd.DataFrame(data={'full_name':full_name, 'rating':rating, 'total_ratings':total_ratings, 'landmark_category':landmark_cat, 'description':description, 'address':address, 'hours':hours})

In [ ]:
HL.head()

<img src="t_dataRaw.png">

---

Cuando se ordena como un DataFrame de Pandas, aunque lo que se vea de los primeros registros parezca poca información en realidad en cada registro de la columna hours lo que se podría encontrar es algo como lo que se observa en imagen siguiente. Sin embargo toda esta información como se visualizará a continuación es información de la actividad del establecimiento a cada hora durante cada día de actividad. Esto quiere decir que si se tiene información de un establecimiento que está abierto seis días a la semana con un horario de 6 de la mañana a 23 horas toda esta información no representa, en el sentido practico, un dato que aporte al análisis. Por lo tanto desde la perspectiva del analista fue conveniente proponer que esta información se comprimiera de alguna manera haciendo la media de la actividad en el establecimiento. Lo anterior se podrá observar con el transcurso de este Notebook.

<img src="t_colHour_dataRaw.png">

In [ ]:
(list(HL[HL.full_name == 'MF Flomart'].hours))

In [ ]:
def visit_planner(place_name) :

    # first we make an empty dataframe out of a list of days as index and a list of hours as columns :'Sunday', 
    place_name = pd.DataFrame(index=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], \
    #place_name = pd.DataFrame(index=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],  \
                              columns=['06','07','08','09','10','11','12','13','14','15','16','17','18','19','20',
                                       '21','22','23'])    
    
    # and then we return that dataframe
    return place_name

<img src="empty_table.png">

Para cada establecimiento se tendrá que crear una tabla vacía con los días de actividad y las horas activas. Lo anterior podrían ser de cinco a siete días y siempre serán diez y ocho horas de actividad, lo que variaría en este caso serian los rangos de las horas de actividad pudiendo comenzar más temprano en algunos casos o más tarde en otros. Por lo tanto aquí es donde reside la dificultad de este proceso, pues según para cada caso que lo requiera el horario y los días de atención de cada establecimiento se debería crear la tabla adecuada.

In [ ]:
table = visit_planner("t_table")

# we need to remove the row for Sunday before proceeding
table.drop('Saturday', axis=0, inplace=True)

for hour in list(table):
    table[hour] = re.findall('.{0,29}hora: '+str(hour)+'..', str(list(HL[HL.full_name=='MF Flomart'].hours)))
    table[hour] = [ re.findall('\d+', str(i))[0] for i in tall2[hour] ]
    table[hour] = table[hour].astype(int)

plt.figure(figsize=(15,7))
sns.heatmap(table, cmap='PuBu', linewidths=0.8, annot=True, annot_kws={'fontsize':8, 'alpha':0.8}, fmt='d', square=True,
           cbar=False)

plt.xticks(np.arange(18), list(tall2))
plt.title("Popular times visiting MF Flomart\n", weight='semibold')

# this bit of code ensures the heatmap will not show up truncated with some versions of matplotlib
b, t = plt.ylim()
b += 0.5
t -= 0.5
plt.ylim(b, t);

<img src="filled_table.png">

Si todo es correcto, uno podría encontrar una tabla rellenada como la se observa en la parte superior, y de manera gráfica como se observa en la figura siguiente (la tabla gráfica solo es útil para darse una idea de la distribución a lo largo de una semana)

<img src="show_ftable.png">

La idea de esto es conseguir de la tabla anterior, lo que era la tabla rellenada con los valores a cada hora, una tabla **.describe()**:

<img src="describe_table.png">

Y aún cuando ya se tuviera lo anterior, la idea final es comprimir esta información haciendo la media de la anterior tabla:

<img src="reduced_described_table.png">

Hasta aquí se ha explicado como se obtuvieron los datos de cada consulta que se fueron realizando en google Maps de cada registro en los datos que tenía Eixos. Además se ha explicado un poco de la idea que se tiene para enriquecer más cada registro de este data set que se obtuvo de Google haciendo uso de la columna hours.

---

# GENERAR MAS INFORMACIÓN A PARTIR DE LA COLUMNA HOURS

Hasta esta parte del proceso se asume que ya se tiene toda la información que se pudo obtener de las consultas realizadas a Google maps. Se puede utilizar un parámetro, ```glob``` para obtener una lista con los ficheros ```csv``` que están disponibles.

In [4]:
# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
csv_files = glob.glob('*.csv')
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(csv_files)

['hfull_stcl_ldmks_attem19798_succ6095.csv', 'hfull_stcl_ldmks_attem33872_succ10300.csv', 'hfull_stcl_ldmks_attem35886_succ1033.csv', 'hfull_stcl_ldmks_attem36514_succ424.csv', 'hfull_stcl_ldmks_attem38201_succ559.csv', 'hfull_stcl_ldmks_attem38849_succ287.csv', 'hfull_stcl_ldmks_attem39701_succ256.csv', 'hfull_stcl_ldmks_attem41723_succ662.csv', 'hfull_stcl_ldmks_attem45541_succ917.csv', 'hfull_stcl_ldmks_attem47051_succ623.csv', 'hfull_stcl_ldmks_attem50323_succ1553.csv', 'hfull_stcl_ldmks_attem50969_succ240.csv', 'hfull_stcl_ldmks_attem51465_succ151.csv', 'hfull_stcl_ldmks_attem53551_succ706.csv', 'hfull_stcl_ldmks_attem59786_succ1368.csv', 'hfull_stcl_ldmks_attem63950_succ1057.csv', 'hfull_stcl_ldmks_attem65546_succ463.csv', 'hfull_stcl_ldmks_attem69434_succ503.csv', 'hfull_stcl_ldmks_attem69894_succ176.csv', 'table_of_MEANS_googleData.csv']


In [4]:
csv_files = ['hfull_stcl_ldmks_attem19798_succ6095.csv',
             'hfull_stcl_ldmks_attem33872_succ10300.csv',
             'hfull_stcl_ldmks_attem35886_succ1033.csv',
             'hfull_stcl_ldmks_attem36514_succ424.csv',
             'hfull_stcl_ldmks_attem38201_succ559.csv',
             'hfull_stcl_ldmks_attem38849_succ287.csv',
             'hfull_stcl_ldmks_attem39701_succ256.csv',
             'hfull_stcl_ldmks_attem41723_succ662.csv',
             'hfull_stcl_ldmks_attem45541_succ917.csv',
             'hfull_stcl_ldmks_attem47051_succ623.csv',
             'hfull_stcl_ldmks_attem50323_succ1553.csv',
             'hfull_stcl_ldmks_attem50969_succ240.csv',
             'hfull_stcl_ldmks_attem51465_succ151.csv',
             'hfull_stcl_ldmks_attem53551_succ706.csv',
             'hfull_stcl_ldmks_attem59786_succ1368.csv',
             'hfull_stcl_ldmks_attem63950_succ1057.csv',
             'hfull_stcl_ldmks_attem65546_succ463.csv',
             'hfull_stcl_ldmks_attem69434_succ503.csv',
             'hfull_stcl_ldmks_attem69894_succ176.csv']
list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y el resultado final será la lista dataframes

for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)

df_concat = pd.concat(list_data,ignore_index=True)
df_concat = df_concat.drop_duplicates().reset_index(drop=True)

Despues con un ciclo se puede ir formado una lista con los elementos de la lista para que de último se pueda concatenar toda la lista. Como ultimo paso es importante retirar los duplicados. Así es como se obtiene el dataset definitivo de todas las consultas realizadas.

In [35]:
df_concat.shape

(21526, 11)

Como se ve anteriormente los registros únicos son 21526, aunque los que se obtuvieron fueron aproximadamente 22mil y un poco más.  
Luego al tener listo en un solo DataFrame todos los registros se crea una nueva tabla vacía donde se guardará la información que se generará por cada registro que cumpla con las condiciones.

In [6]:
comp_df = pd.DataFrame(index = df_concat.index.values, \
                       columns=('count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'))

En la siguiente función lo que se hace es declarar tablas para los cuatro casos de horarios que se considerarán:
* de 24h a 17h
* de 04h a 21h
* de 05h a 22h
* de 06h a 23h  

Todas la tablas asumen al inicio que si se invocan se crearán con los siete dias y si se tiene que eliminar un día, pues así sera.

In [7]:
def visit_planner0_17(place_name) :
    # first we make an empty dataframe out of a list of days as index and a list of hours as columns :
    place_name = pd.DataFrame(index=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], \
                              columns=['24','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17'])    
    return place_name

def visit_planner4_21(place_name) :
    # first we make an empty dataframe out of a list of days as index and a list of hours as columns :
    place_name = pd.DataFrame(index=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], \
                              columns=['04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21'])    
    return place_name

def visit_planner5_22(place_name) :
    # first we make an empty dataframe out of a list of days as index and a list of hours as columns :
    place_name = pd.DataFrame(index=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], \
                              columns=['05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22'])    
    return place_name


def visit_planner6_23(place_name) :
    # first we make an empty dataframe out of a list of days as index and a list of hours as columns :
    place_name = pd.DataFrame(index=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], \
                              columns=['06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23'])    
    return place_name

---

Función para eliminar formato:  
    ```'Nivel de ocupación: 17\\\\\\\\xa0% (hora: 18:00)``` -> X  
    ```'Nivel de ocupación: 17\\\\\\\\xa0% (hora: 18)``` -> OK

In [8]:
def supZeros(chnk):
    line = str(list(df_concat[df_concat.index == chnk].hours))
    df_concat['hours'][chnk] = re.sub('\:00\)', ')', line)[2:-2]

---

Función para cambiar ciertos registros donde los siete dias de la semana pero seis dias a las 05 h hasta las 22 h y un dia a las 06 h hasta las 23h.  
Lo que se hacia aqui era cambiar el registro con la hora 23 por el registro de la hora 05.

In [9]:
def chgTtbyFi(chnk):
    lsd = []
    sd = str(df_concat['hours'][chnk]).split(',')
    for i in sd:
            if bool(re.findall('\: 23\)', i)) == True:
                sd[sd.index(i)] = ' Nivel de ocupación:  0\\xa0% (hora: 05).' 
                print(i, chnk)
    df_concat['hours'][chnk] = sd[2:-2]

---

Función para solucionar registros de establecimeintos que se capturaban justo cuando estos estaban abiertos y se mostraba en tiempo real la afluencia de personas y por otro lado se tenía la afluencia habitual de los establecimientos. Este fue un serio problema pues si no se solucionaba se perdía esta información, información que si estaba pero no en le formato que se esperaba.  
  
La situación era este tipo de formato:  
`...`  
`" 'Nivel de ocupación: 54\\\\xa0% (hora: 24).'",`  
`" 'Nivel de ocupación: 39\\\\xa0% (hora: 01).'",`  
`" 'Nivel de ocupación actual: 0\\\\xa0%. Nivel de ocupación habitual: 27\\\\xa0%.'",`   
`" '0%'",`  
`' \'Nivel de ocupación: 17\\\\xa0% (hora: 03)`  
`...`  
Lo que se desea es algo como esto:  
`...`  
`" 'Nivel de ocupación: 54\\\\xa0% (hora: 24).'",`  
`" 'Nivel de ocupación: 39\\\\xa0% (hora: 01).'",`  
`" 'Nivel de ocupación: 27\\\\xa0% (hora: 02).'",`   
`' \'Nivel de ocupación: 17\\\\xa0% (hora: 03)`  
`...`  
  
Esto es lo que se encarga de hacer esta función, corregir este formato y eliminar ese elemento que esta de sobra.

In [10]:
def cambioValSpl(chnk):
        sd = str(df_concat['hours'][chnk]).split(',')
        lst = []
        lsd = []
        for i in sd:
            lst.append(len(i))
            if len(i) < 40:
                lsd.append(sd.index(i))
        posMax = lst.index(max(lst))
        sd = str(df_concat['hours'][chnk]).split(',')
        lst=[]
        i = str(re.findall('.{0,10}habitual.{0,11}', str(list(df_concat[df_concat.index==chnk].hours))))[2:-2]
        ii = int(re.findall('\d+',i)[0])
        isw = int(re.findall('\d+',sd[posMax-1])[2])+1
        #'Nivel de ocupación: 29\\xa0% (hora: 16).'
        if isw < 10:
            sd[posMax] = ' \'Nivel de ocupación: '+str(ii)+'\\xa0% (hora: 0'+ str(isw) +').'
        else: 
            sd[posMax] = ' \'Nivel de ocupación: '+str(ii)+'\\xa0% (hora: '+ str(isw) +').'
        sd = np.delete(sd,lsd)
        df_concat['hours'][chnk] = sd

---

Aqui es donde se ejecuta la funcio del cambio de formato, pero solo para los que tengan la palabra clave **habitual**, lo que hace es enviar el valor del indice de este registro y a partir de alli hace todo el proceso necesario para cambiar el formato de ese registro, manipulando solamente aquello que esta mal.

In [11]:
dfConcat_inx = df_concat.index.values
for i in dfConcat_inx:
    if bool(re.search("habitual.",str(list(df_concat[df_concat.index == i].hours)))) == True:
        cambioValSpl(i)

C:\Users\alter\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


---

Aqui se eliminan los:   
```['30% concurrido a la(s) 09:00',
 '26% concurrido a la(s) 09:00',
 '26% concurrido a la(s) 09:00',
 '43% concurrido a la(s) 09:00',
 '52% concurrido a la(s) 09:00',
 '39% concurrido a la(s) 09:00']```  

Por un formato asi:  
```['30% concurrido a la(s) 09',
 '26% concurrido a la(s) 09',
 '26% concurrido a la(s) 09',
 '43% concurrido a la(s) 09',
 '52% concurrido a la(s) 09',
 '39% concurrido a la(s) 09']```  
   
Esto es porque asi se decicio por convención en este análisis, además la mayoria de los registros tenían el formato sin los *:00*, entoces se opto por dejarlo asi. Por ultimo como razon principal fue porque es mas facil manejar este tipo de formato al momento de rellenar la tabla con los valores de afluencia por dia/hora.

In [12]:
dfConcat_inx = df_concat.index.values
for i in dfConcat_inx:
    
    if bool(re.search("\:00",str(list(df_concat[df_concat.index == i].hours)))) == True:
        supZeros(i)

C:\Users\alter\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


---

Despues de limpiar y tratar que los datos fueran lo más homogéneos posibles, aquí se presenta la parte más importante del código, pues aquí se decidirá a que tabla se llamará, no obstante esto dependerá de una verificación de lo que se halla en el registro. Como se mencionó al inicio hubieron ciertas condiciones que se tomaron en cuenta. Como primer paso lo que se realiza es crear dos listas en un array numpy, por un lado los indices de cada uno de los registros en el data set y por otro lado la longitud de los elementos que estan almacenados en esos registros, con esto podremos darnos una idea de que es lo que tenemos guardado. Algunas pautas esenciales fueron:  

Condiciones esenciales:  
* Se asume que un día de trabajo tendra al menos 18 horas de actividad
* se asume que para el analisis solmamente se utilizará la información de lo establecimientos que esten abiertos cinco dias o mas.
* Teniendo en cuenta lo anterior, logicamente el valor mínimo de a longitud de los registros que se analizaran seran los que sean igual o más a 90 elementos. Porque 5\*18=90  

Valores mínimos:  
* SIETE DIA -> 7\*18 = 126  
* SEIS DIAS -> 6\*18 = 108  
* CINCO DIAS -> 5\*18 = 90  

Los valores anteriores por dia representan los valores minimos en cada caso, pues en algunos registros se tiene actividad muchas mas horas pero para este analisis solo seran utiles 18h por dia, por lo tanto aqui es donde entra nuestro proceso de verificación.
Antes de crear esa tabla vacia que nos sera útil, se debe verificar, para esto se contará de forma estrategia el numero de ocurrencias de ciertas horas que se observaron que eran las mas determinantes . Definir estas horas estrategicas se logró despues de analizar mucho tiempo la naturaleza de la información que se encontraba en los  registros de la columna hours. En teoría, se tendría que verificar que para cada caso existan los mismos numeros de ocurrencias de las horas, es decir que de un establecimiento que abre los siete dias de la semana con un intérvalo de 18 h que va de 05 h a 22 h, se deben encontrar siete registros de la hora 05, siete registros de la hora 06, siete registros de la hora 07... asi sucesivamente hasta por ultimo verificar la hora 22. Lo anterior aunque sea lo mas correcto no es lo mas eficiente, por lo tanto se buscaron ciertas horas especificas para dar con la tabla correcta que fuera necesaria para cada caso.  
Cuando ya se haya verificado que tipo de informacion es la que existe dentro del registro entonces se llama a la funcion que crea la tabla mas adecuada. Luego a menos que sea un estableciemiento que este abierto los siete dias de la semana se crea la tabla y listo, de lo contrario se tendra que que eliminar los dias que sean necesarios, siendo posibles los casos de eliminar el domingo o el sabado y el domingo.
Ahora si, cuando ya se tiene la tabla correcta, creada y vacia es cuando se rellena. En el paso de verificacion se habían utilizado algunas funciones regex, sobre todo para encontrar patrones detro de estos bloques de información. Sin embargo, en este caso si que se se utilizan de manera que se vayan guardando y asigano a cada espacio que le corresponde detro de la tabla.  
Luego que ya se tiene esa tabla con los valores, creamos un Dataframe vacio al que se le asigará la tabla *describe* de la anterior tabla pero transpuesta y cuando ya esta lista es a esta tabla a la que se le aplica la función *mean()* y en este punto es donde tenemos la informacion comprimida del registro. Lo que queda es guaradar estas nuevas columnas con un registro de la tabla comp_df () fue la tabla vacia que declaramos en pasos anteriores con los indices de la tabla df_concant. De ese registro en específico se guaradará la media de la información las siguientes columnas:  

   * count 	
   * mean  
   * std 	
   * min 	
   * 25% 	
   * 50% 	
   * 75% 	
   * max  

Es asi como se comprime toda la información y como al final toda esta tabla con informacion de establecimeintos que estan abiertos 5, 6 o 7 dias con un itnervalo de 18 h se pueden guardar a la par de la tabal df_concat, la tabla donde encontramos informacion de los registros como el nombre, su direcion, las reseñas y valoraciones entro otras cosas. Es asi como finalmente se ha enriquecido este dataset.

In [13]:
HL_inx = df_concat.index.values
cntPer= df_concat.hours.apply(lambda x: len(str(x).split(',')))
#cntPer = df_concat.hours.apply(lambda x: len(x.split(',')))
arrInf = np.column_stack((HL_inx,cntPer))
vac = []
iver = []
#sev = False

# visit_plannerD visit_plannerC visit_plannerS


for (i, j) in arrInf:
    if j >= 90:
        
        sev = False
        six = False
        fiv = False
        
        #SIETE DIAS
        if (len(re.findall('.{0,36}04'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}21'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 ):
            #print(i, j,'tabla de seis dias de 05 a 10')
            vac.append(i)
            t_table = visit_planner4_21("temp_table")
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1] 
            sev = True
            
        if (len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and sev == False):
            #print(i, j,'tabla de seis dias de 05 a 10')
            vac.append(i)
            t_table = visit_planner5_22("temp_table")
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1] 
            sev = True
            
        if (len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}23'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and sev == False):
            #print(i, j,'tabla de seis dias de 06 a 11')
            vac.append(i)
            t_table = visit_planner6_23("temp_table")
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]   
            sev = True
                    
        if (len(re.findall('.{0,36}24'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}03'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and \
            len(re.findall('.{0,36}17'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and 
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 7 and sev == False):
            #print(i, j,'tabla de seis dias de 06 a 11')
            vac.append(i)
            t_table = visit_planner0_17("temp_table")
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]           
            sev = True
            
        #SEIS DIAS            
        if (len(re.findall('.{0,36}04'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}21'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and sev == False ):
            #print(i, j,'tabla de seis dias de 05 a 10')
            vac.append(i)
            t_table = visit_planner4_21("temp_table")
            t_table.drop('Sunday', axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            
            six = True

        if (len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and sev == False and six == False ):
            #print(i, j,'tabla de seis dias de 05 a 10')
            vac.append(i)
            
            t_table = visit_planner5_22("temp_table")
            t_table.drop('Sunday', axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            
            six = True
            
        if (len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}23'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and sev == False and six == False ):
            #print(i, j,'tabla de seis dias de 06 a 11')
            vac.append(i)
            
            t_table = visit_planner6_23("temp_table")
            t_table.drop('Sunday', axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            
            six = True
            
        if (len(re.findall('.{0,36}24'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}03'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}17'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 6 and sev == False and six == False ):
            #print(i, j,'tabla de seis dias de 06 a 11')
            vac.append(i)
            t_table = visit_planner0_17("temp_table")
            t_table.drop('Sunday', axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            
            six = True
            
        #CINCO DIAS
        if (len(re.findall('.{0,36}04'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}21'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and sev == False and six == False):
            #print(i, j,'tabla de seis dias de 05 a 10')
            vac.append(i)
            t_table = visit_planner4_21("temp_table")
            t_table.drop(['Sunday','Saturday'], axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            fiv = True
            
        if (len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and sev == False and six == False and fiv == False):
            #print(i, j,'tabla de cinco dias de 05 a 10')
            vac.append(i)
            t_table = visit_planner5_22("temp_table")
            t_table.drop(['Sunday','Saturday'], axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            fiv = True
            
        if (len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}23'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}22'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and sev == False and six == False and fiv == False):
            #print(i, j,'tabla de cinco dias de 06 a 11')
            vac.append(i)
            t_table = visit_planner6_23("temp_table")
            t_table.drop(['Sunday','Saturday'], axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
            fiv = True
        if (len(re.findall('.{0,36}24'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}03'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}17'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}05'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}06'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and \
            len(re.findall('.{0,36}07'+'\)', str(list(df_concat[df_concat.index == i].hours))))== 5 and sev == False and six == False and fiv == False):
            vac.append(i)
            t_table = visit_planner0_17("temp_table")
            t_table.drop(['Sunday','Saturday'], axis=0, inplace=True)
            for hour in list(t_table):
                
                t_table[hour] = re.findall('.{0,36}'+hour+'\)', str(list(df_concat[df_concat.index == i].hours)))
                t_table[hour] = [ re.findall('\d+', str(j))[0] for j in t_table[hour] ]
                t_table[hour] = t_table[hour].astype(int)
            table_des = pd.DataFrame()
            table_des = pd.DataFrame((t_table.describe()[:]).mean(axis=1)).T
            comp_df.iloc[i] = table_des.iloc[-1]
        if(fiv == True or six == True or sev == True):
            iver.append(i)
len(vac)

5470

In [14]:
comp_df

,count,mean,std,min,25%,50%,75%,max
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,18.9444,8.10061,10.5556,14.1667,17.5,21.9444,30.5556
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
21521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finalmente aqui se une la información original de Google Maps de los registros y la información que surge de la columna hours. Es importante mencionar que aqui solo se contemplo los valores de la media de la tabla describe() de cada registro, pero podrian obtenerse otros valores que podrian resultar interesantes como la sumatoria de los porcentajes de afluencia o solamente tener en cuenta ciertos dias de la semana o que la tabla describre() no fuera sobre las horas sino sobre los dias, etc. En realidad se podrian obtener mas datos estadisticos de los registros de la columna **hour**, pero eso queda como un comentario, pues en este proyecto no se abordarán más de esas combinaciones.

In [15]:
final_GoogleMaps = pd.merge(df_concat, comp_df, left_index=True, right_index=True, how='outer')
final_GoogleMaps

,full_name,rating,total_ratings,landmark_category,description,address,phone,website,hours,latitude,longitude,count,mean,std,min,25%,50%,75%,max
0,Tallers Garcés,"4,1",(10),Taller de automóviles,NaN,"Carrer de Rocafort, 78, 08015 Barcelona",934 23 10 93,NaN,[],41.379371,2.153754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Taller Antón Solé,NaN,NaN,NaN,NaN,"Plaça de las Navas, 10, 08004 Barcelona",934 23 45 63,NaN,[],41.374035,2.158775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Auto Carburacion e Inyeccion,"5,0",(2),Taller de reparación de vehículos todoterreno,NaN,"C/ d'Entença, 20, 08015 Barcelona",933 25 46 94,NaN,[],41.376014,2.155615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MF Flomart,"4,9",(35),Tienda de repuestos para automóviles,NaN,"Passeig de Montjuïc, 30, 08004 Barcelona",934 41 13 48,NaN,"['Nivel de ocupación: \xa0% (hora: ).', 'Nivel...",41.372421,2.171510,5,18.9444,8.10061,10.5556,14.1667,17.5,21.9444,30.5556
4,Talleres Espuña,"4,6",(10),Taller de reparación de automóviles,NaN,"Passeig de Montjuïc, 68, 08004 Barcelona",934 41 48 10,NaN,[],41.371756,2.167712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21521,De Pata Negra,"3,7",(264),Bar,NaN,"Plaça de las Navas, 7, 08004 Barcelona, España",NaN,m.facebook.com,[],41.373856,2.159229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21522,Malabida,"4,5",(171),Bar restaurante,NaN,"Carrer de Blai, 63, 08004 Barcelona, España",+34 931 75 81 79,malabida.business.site,"['Nivel de ocupación: 0\\xa0% (hora: 04).', 'N...",41.374590,2.161894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21523,Bodega 1900,"4,4",(990),Bar de tapas,NaN,"Carrer de Tamarit, 91, 08015 Barcelona, España",+34 933 25 26 59,elbarri.com,[],41.375552,2.156562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21524,Vinoteca San Antoni By Wine Palace,"4,5",(68),Bodega,NaN,"Carrer del Comte Borrell, 30, 08015 Barcelona,...",+34 935 39 40 02,winepalace.es,"['Nivel de ocupación: \\xa0% (hora: ).', 'Nive...",41.376737,2.163638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
final_GoogleMaps.to_csv('table_of_MEANS_googleData.csv', index=False)

In [3]:
import pandas as pd
import numpy as np

In [5]:
final_GoogleMaps = pd.read_csv('table_of_MEANS_googleData.csv')

In [6]:
final_GoogleMaps['id'] = final_GoogleMaps.index

In [7]:
final_GoogleMaps.to_csv('table_of_MEANS_googleData.csv', index=False)